# Color-filled Gridded Output

This notebook creates an example of overlaying color-filled contours and regular contours over the CONUS.

### Import Needed Packages

In [ ]:
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from metpy.plots import declarative
from metpy.units import units
import xarray as xr

### Get Data

In [ ]:
# Set the date/time of the model run
date = datetime(2022, 2, 10, 0)

# Remote access to the dataset from the UCAR site
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                     f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55)).metpy.parse_cf()

### Plot Data

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds
cntr.field = 'Pressure_reduced_to_MSL_msl'
cntr.level = None
cntr.time = plot_time
cntr.contours = list(range(0, 2000, 4))
cntr.clabels = True
cntr.plot_units = 'hPa'

# Set attributes for plotting filled contours
cfill = declarative.FilledContourPlot()
cfill.data = ds
cfill.field = 'Temperature_height_above_ground'
cfill.level = 2 * units.m
cfill.time = plot_time
cfill.contours = list(range(-38, 103, 2))
cfill.colormap = 'coolwarm'
cfill.colorbar = 'horizontal'
cfill.plot_units = 'degF'

# Set attributes for plotting wind barbs
barbs = declarative.BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_height_above_ground',
               'v-component_of_wind_height_above_ground']
barbs.level = 10 * units.m
barbs.skip = (2, 2)
barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'MSLP (hPa) and TMPF at {plot_time} by KHG'
panel.plots = [cntr, cfill, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()